# Hotel's Reviews Sentiment Analysis
## Gathering Data
First step is to gather the data from DB.
```
select translation_title, translation_text, score
from hotel_reputation;
```

We then load the file to start working with. I chose JSON instead of CSV because we are working with tests, which have commas and quotes and other symbols as well, so it would be difficult for CSV to know how to separate the columns without having conflicts with the actual contents. Also, JSON is easier to look at for humans. The downside is that it weights roughly 20 MB more and tjus it's also slightly slower to load and occupies slightly more memory when loaded.

In [ ]:
import pandas as pd

reviews_df = pd.read_json('reviews.json')
pd.set_option('display.max_colwidth', None)
print(reviews_df.shape)
print(reviews_df.head())

## Data Processing
### Combining Title and Text
Next we will combine the title and text columns into a single one, so we can work with the text as a whole.

In [ ]:
def combine_texts(title, text):
    if pd.isna(title) and pd.isna(text):
        return ''
    elif pd.isna(text) or text.strip() == '':
        return title
    elif pd.isna(title) or title.strip() == '':
        return text
    else:
        return f"{title} {text}"

reviews_df['combined_text'] = reviews_df.apply(lambda row: combine_texts(row['translation_title'], row['translation_text']), axis=1)
reviews_df.drop(columns=['translation_title', 'translation_text'], inplace=True)
print(reviews_df.head())

### Cleaning Data

#### Removing rows with combined_text empty
Some rows didn't have nor title nor text, so we have `null` values in the `combined_text` column:

In [ ]:
print('Rows with combined_text empty:')
print(len(reviews_df[reviews_df['combined_text'] == '']))

We will remove these rows, as they don't bring any value to our analysis.

In [ ]:
print('Length before removing rows with combined_text empty:', len(reviews_df))
reviews_df = reviews_df[reviews_df['combined_text'] != '']
print('Length after removing rows with combined_text empty:', len(reviews_df))

#### Fixing rows with scores not boxed in [1, 2, 3, 4, 5]
Some rows had scores not boxed in [1, 2, 3, 4, 5]:

In [ ]:
print('Scores not boxed in [1, 2, 3, 4, 5]:')
print(len(reviews_df[~reviews_df['score'].isin([1, 2, 3, 4, 5])]))
print('-' * 50)
print('Value counts:')
print(reviews_df['score'].value_counts())

We will not remove this rows, as their data is still useful, instead, we will think of a way to box them in one of the 5 categories.
A side investigation demonstrated that the best strategy is to ceil them (refer to the previous Google collab notebooks).

In [ ]:
import numpy as np

reviews_df['score'] = reviews_df['score'].apply(np.ceil)
print('Scores not boxed in [1, 2, 3, 4, 5]:')
print(len(reviews_df[~reviews_df['score'].isin([1, 2, 3, 4, 5])]))
print('-' * 50)
print('Value counts:')
print(reviews_df['score'].value_counts())

#### Fixing rows with trailing and leading spaces
We have rows with trailing and leading spaces in the `combined_text` column:

In [ ]:
print('Rows with trailing and leading spaces in the combined_text column:')
print(len(reviews_df[reviews_df['combined_text'].str.contains('^\s+|\s+$')]))

We will remove these spaces.

In [ ]:
reviews_df['combined_text'] = reviews_df['combined_text'].str.strip()
print('Rows with trailing and leading spaces in the combined_text column:')
print(len(reviews_df[reviews_df['combined_text'].str.contains('^\s+|\s+$')]))

## Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

X = reviews_df['combined_text']
y = reviews_df['score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Data Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

## Model Training

In [47]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1500)
model.fit(X_train_vectorized, y_train)


C:\Users\Alvar\OneDrive\Escritorio\ai-experiment\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1500)

## Model Evaluation

In [48]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

x_pred = model.predict(X_train_vectorized)
y_pred = model.predict(X_test_vectorized)
train_accuracy = accuracy_score(y_train, x_pred)
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')
print(f'Testing Accuracy: {test_accuracy * 100:.2f}%')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Training Accuracy: 91.68%
Testing Accuracy: 87.97%
Confusion Matrix:
[[ 1221   131    58    43     9]
 [  107  1753   188   110    18]
 [   70   182  5328   880   145]
 [   26    55   582 24259  2619]
 [    9    15   170  3496 32610]]
Classification Report:
              precision    recall  f1-score   support

         1.0       0.85      0.84      0.84      1462
         2.0       0.82      0.81      0.81      2176
         3.0       0.84      0.81      0.82      6605
         4.0       0.84      0.88      0.86     27541
         5.0       0.92      0.90      0.91     36300

    accuracy                           0.88     74084
   macro avg       0.86      0.85      0.85     74084
weighted avg       0.88      0.88      0.88     74084

